In [ ]:
import os
import csv
import requests
import json
import math
import urllib

In [ ]:
BranchCommits= None
team_members= None
names= None
Branches = None
memberIDToName =None
projects_url= None
teams_url = None
commits_url =None
issues_search_url=None
issues_url = None
pulls_search_url = None
pulls_url = None
milestones_url=None 
branches_url = None

In [ ]:
'''TO BE CHANGED'''

instructor_id = ""
instructor_name = ""
ta_id = ""
org = ""
repo = ""


url_headers = {
                "Accept" : "application/vnd.github.cloak-preview",
            }
project_api_headers= {"Accept":"application/vnd.github.inertia-preview+json"}
user =""
token = ""
parameters={"state":"all"}
teams_url=None
team_members=None
def makeURLs(organisation,repo):
    projects_url = "https://api.github.com/repos/"+organisation+"/"+repo+"/projects"
    teams_url = "https://api.github.com/orgs/" +organisation+"/teams"
#     commits_url = "https://api.github.com/search/commits?q=repo:"+organisation+"/"+repo+ " author-date:>1990-01-01"
    commits_url =   "https://api.github.com/repos/" + organisation + "/" +repo+ "/commits?sha="
    issues_search_url = "https://api.github.com/search/issues?q= repo:"+organisation+"/"+repo
    issues_url = "https://api.github.com/repos/" + organisation + "/" +repo+ "/issues"
    pulls_search_url = "https://api.github.com/search/issues?q= repo:" + organisation+"/"+repo+" type:pr"
    pulls_url = "https://api.github.com/repos/" + organisation + "/" +repo+ "/pulls"
    milestones_url = "https://api.github.com/repos/" + organisation + "/" +repo+ "/milestones"
    branches_url = "https://api.github.com/repos/"+organisation +"/" + repo + "/branches?per_page=100"
    return projects_url,teams_url,commits_url,issues_search_url,issues_url,pulls_search_url,pulls_url,milestones_url, branches_url

projects_url,teams_url,commits_url,issues_search_url,issues_url,pulls_search_url,pulls_url,milestones_url, branches_url= makeURLs(org,repo)


In [ ]:

# /orgs/:org/teams/:team_slug/members
def getTeamMembers(teamUrl=teams_url, Headers=url_headers,Auth=(user,token),instructorID=instructor_id,
                  instructorName=instructor_name,ta=ta_id):
    teamMembers={}
    memberIDToName = {}
    response = requests.get(teamUrl,headers=Headers,auth=Auth)
    result = response.json()
#     print(json.dumps(result,indent=4))
    for team in result:
#         print(json.dumps(team,indent=4))
        url = "https://api.github.com/orgs/"+ org+ "/teams/"+team["slug"]+"/members"
#         print(url)
        members = requests.get(url,headers=Headers,auth=Auth)
#         print(members.text)
        members = members.json()
#         print(json.dumps(members,indent=4))
        for member in members:
#             print(json.dumps(member,indent=4))
            response = requests.get(member["url"],headers=Headers,auth=Auth)
            result = response.json()
#             print(json.dumps(result,indent=4))
            teamMembers[member["login"]] = team["name"]
            if result["name"] != None:
                memberIDToName[member["login"]] = result["name"]
            else:
                memberIDToName[member["login"]] = "not specified"
        
    teamMembers[instructorID] = "Instructor"
    teamMembers[instructorName] = "Instructor"
    teamMembers[ta] = "teaching assistant"
#             print(member["login"],teamMembers[member["login"]])
    
    print(teamMembers)
    print(memberIDToName)
    return teamMembers,memberIDToName

team_members,memberIDToName = getTeamMembers()


In [ ]:
def get_branches(url=branches_url,Headers=url_headers,Auth=(user,token)):
    branches = []
    response = requests.get(url,headers=Headers,auth=Auth)
    result = response.json()
    for item in result:
        branch = item["name"]
        branches.append(branch)
    print(f"total branches: {len(branches)}")
    print(branches)
    return branches
Branches = get_branches()

In [ ]:
'''COMMITS START HERE'''

def get_total_pages(search_url,Headers=url_headers,Auth=(user,token)):
    response = requests.get(search_url, headers=Headers,auth=Auth)
    result = response.json()
    total = result["total_count"]
    print("Github returns 30 items per page.....")
    print(f"total items found: {total}")
    pages = math.ceil(total/30)
    print(f"(on) total pages: {pages}")
    return pages

# commitPages = get_total_pages(commits_url)

In [ ]:
def getNextURL(url,Params={},Headers=url_headers,Auth=(user,token)):
    response = requests.get(url,headers=Headers,auth=Auth,params=Params)
#     print(response.text)
#     print(response.headers)
    link = response.headers["Link"]
#     print(link)
#     total_count = response.json()["total_count"]
#     print(f'total count = {total_count}')
    links = link.split(",")
    linkHash = {}
    for item in links:
        array = item.split(";")
        linkHash[array[1].replace('"',"").replace(" ","")] = array[0].replace("<","").replace(">","")
    
    return(linkHash["rel=next"])

    
# getNextURL(commits_url)



In [ ]:
# print(f'total pages to process: {commitPages}')
def get_All_URLs(url,limit=1000,params={},Headers=url_headers):
    urls = []
    urls.append(url)
    for x in range(1,limit):
        try:
            url = getNextURL(urls[x-1],params,Headers)
#             print(url)
            if url == "" or url== None:
                break
            else:
                urls.append(url)
        except:
            break
            
        
    print(f"total pages: {len(urls)}")
#     print(urls)
    return urls

# all_commit_urls = get_All_URLs(link)

In [ ]:

def get_branch_commit_urls(URL=commits_url,branches=Branches,Headers=url_headers,Auth=(user,token)):
    branchCommits = {}
    for branch in branches:
        urls = []
        url = URL + urllib.parse.quote(branch)
        branch_pages = get_All_URLs(url)
        for page in branch_pages:
            response = requests.get(page,headers=Headers,auth=Auth)
            result = response.json()
#             print(json.dumps(result,indent=4))
            for commitItem in result:
                commitURL = commitItem["url"]
                urls.append(commitURL)
        branchCommits[branch] = urls
        print(f"branch: {branch}, total commits: {len(urls)}")
    return branchCommits

BranchCommits = get_branch_commit_urls()



In [ ]:
master = BranchCommits["master"]
print(len(master))
masterBranch = {}
masterBranch["master"] = master



In [ ]:
def get_commit_urls(urls, Headers=url_headers):
    count = 0
    dataLinks = []
    print(f'total URLs: {len(urls)}')
    for url in urls:
        response = requests.get(url,headers=Headers,auth=(user,token))
        result = response.json()
        items = result["items"]
        for item in items:
#             print(count,item["url"])
            dataLinks.append(item["url"])
            count+=1
    return dataLinks

# commitLinks = get_commit_urls(all_commit_urls)
# print(f'total commit items to scrape: {len(commitLinks)}')

    

In [ ]:

def get_commit_data(commitBranches=masterBranch,teamMembers=team_members,names=memberIDToName):
    data = []
    for branch in commitBranches.keys():
        commitURLs = commitBranches[branch]
        count = 0
        for url in commitURLs:
            item = {}
            count+=1
            response = requests.get(url,headers=url_headers,auth=(user,token))
            result = response.json()
            item["timestamp"] = result["commit"]["committer"]["date"]
            personID = "n/a"
            key1 = "author"
            if key1 in result.keys() and result[key1] != None:
                key2 = "login"
                if key2 in result[key1].keys() and result[key1][key2] != None:
                    personID = result["author"]["login"]
            else:
                key1 = "login"
                if key1 in result["commit"]["committer"].keys() and result["commit"]["committer"][key1]!=None:
                    personID = result["commit"]["committer"]["login"]
                else:
                    key1 = "author"
                    if key1 in result["commit"].keys() and result["commit"]["author"]["name"]!= None:
                        personID = result["commit"]["author"]["name"]
#                         print(json.dumps(result,indent=4))
                      
                    else:
                        personID = "n/a"

            team = "n/a"
            person = result["commit"]["author"]["name"]
            
            item["personID"] = personID
            item["teamID"] =  team
            item["type"] = "commit"
            item["branch"] = "branch: " + branch
            item["person"] = person
            item["commitMessage"] = result["commit"]["message"]
            item["commitAdditions"] = result["stats"]["additions"]
            item["commitDeletions"] = result["stats"]["deletions"]
            item["commitFilesTouched"] = []

            for file in result["files"]:
                item["commitFilesTouched"].append(file["filename"])
            item["url"] = result["html_url"]
            print(item)
            data.append(item)
        print("\n\n")
        print(f"branch: {branch}, commit items fetched: {count}")
        print("-------------------------------------------------------------------------------------")
        print("\n\n")
    return {"commitData" : data }
commitData = get_commit_data()
print(commitData)

In [ ]:
print(len(commitData["commitData"]))

In [ ]:
def getCommitData(links,teamMembers=team_members):
    data = []
    count=0
    for link in links:
        print("-----------------------------------------------------------------------------------------")
        item = {}
        response = requests.get(link,headers=url_headers,auth=(user,token))
        result = response.json()
#         print(json.dumps(result,indent=4))
        
        item["timestamp"] = result["commit"]["committer"]["date"]
#         print(json.dumps(result,indent=4))

       
        item["personID"] =result["commit"]["committer"]["name"]
        key = "committer"
        if key in result.keys():
            key = "login"
            if result["committer"]!=None and key in result["committer"]: 
                item["personID"] = result["committer"]["login"]
        else:
            key = "author"
            if key in result.keys():
                key = "login"
                if result["author"]!=None and key in result["author"]:
                    item["personID"] = result["author"]["login"]
        key = item["personID"]
        if key in teamMembers:
            item["teamID"] = teamMembers[item["personID"]]
        else:
            item["teamID"] = "n/a"
        item["type"] ="commit"
        item["author"] = result["commit"]["author"]["name"]
        item["commitMessage"] = result["commit"]["message"]
        item["commitAdditions"] = result["stats"]["additions"]
        item["commitDeletions"] = result["stats"]["deletions"]
        item["commitFilesTouched"] = []

        for file in result["files"]:
            item["commitFilesTouched"].append(file["filename"])
        print(item)
        data.append(item)
    return {"commitData" : data }


# commitData = getCommitData(commitLinks)


In [ ]:
'''CREATE MILESTONES'''


def getMilestonesData(teamMembers=team_members,url=milestones_url, Headers=url_headers,Auth=(user,token)):
    response = requests.get(url,headers=url_headers,auth=(user,token))
    milestones = response.json()
    milestoneIDs = {}
    print(f"Total milestones for the repo: {len(milestones)}")
#     print(json.dumps(result,indent=4))
    createMilestoneList = []
    for milestone in milestones:
        milestoneIDs[milestone["title"]] = milestone["number"] 
        createMilestone = {}
        createMilestone["timestamp"] = milestone["created_at"]
        createMilestone["personID"] = milestone["creator"]["login"]
        key = createMilestone["personID"]
        createMilestone["teamID"] ="n/a"
        if key in teamMembers:
            createMilestone["teamID"] = teamMembers[createMilestone["personID"]]
        createMilestone["type"] = "createMilestone"
        createMilestone["title"] = milestone["title"]
        createMilestone["milestoneDescription"] = milestone["description"]
        createMilestone["milestoneID"] = milestone["number"]
        createMilestone["url"] = milestone["html_url"]
        print("------------------------createMilestone-----------------------------------------")
        print(createMilestone)
        createMilestoneList.append(createMilestone)
    
    return {"createMilestone": createMilestoneList},milestoneIDs

milestoneData,milestoneIDs = getMilestonesData()
# print(milestoneIDs)

In [ ]:
'''ISSUES START HERE'''
total_issue_pages = get_total_pages(issues_search_url)
all_issue_URLs = get_All_URLs(issues_url,params=parameters,limit=total_issue_pages)

def separate_issues_from_pulls(urls,params=parameters, Headers=url_headers,Auth=(user,token)):
    issues=[]
    pulls=[]
    print(f'total URLs: {len(urls)}')
    for url in urls:
        response = requests.get(url,headers=Headers,auth=Auth,params=parameters)
        result = response.json()
        for item in result:
            key ="pull_request"
            if key in item:
                pulls.append(item)
            else:
#                 print(json.dumps(item,indent=4))
                issues.append(item)
    print(f"total issues:  {len(issues)}")
    print(f"total pull requests:  {len(pulls)}")
    return issues,pulls

Issues,Pulls = separate_issues_from_pulls(urls=all_issue_URLs)



In [ ]:
def getCheckboxes(text):
    return text.count("[x]") + text.count("[ ]")

def getCheckedCheckboxes(text):
    return text.count("[x]")

In [ ]:
def findIssueProjectBoard(url=projects_url,Headers=project_api_headers,Auth=(user,token)):
    issueInProjectBoard = {}
    response = requests.get(url,headers=Headers,auth=(user,token))
    kanbans = response.json()
#     print(f"Total projects for the repo: {len(kanbans)}")
#     print(json.dumps(kanbans,indent=4))
    for kanban in kanbans:

        projectBoard = []
        boardNumber = kanban["number"]
        boardName = kanban["name"]
        projectBoard.append(boardNumber)
        projectBoard.append(boardName)
#         print(f"Board Number: {boardNumber}, Board Name: {boardName}")
        response = requests.get(kanban["columns_url"],headers=Headers,auth=Auth)
        columns = response.json()
#         print(json.dumps(columns,indent=4))
        for column in columns:
            name = column["name"]
#             print(f"Column: {name}")
            response = requests.get(column["cards_url"],headers=Headers,auth=Auth)
            cards = response.json()
            for card in cards:
                issue = card["content_url"].split("/")[-1]
                issueInProjectBoard[str(issue)] = projectBoard
    return issueInProjectBoard

# issueProjectBoards = findIssueProjectBoard()
# print(issueProjectBoards)

In [ ]:
def getIssueData(issues,milestones,teamMembers=team_members,Headers=url_headers,Auth=(user,token)):
    createIssueList = []
    addLabelToIssueList = []
    assignIssueToPersonList = []
    assignIssueToMilestoneList = []
    commentOnIssueList = []
    tagIssueList = []
    addIssueToKanbanList = []
    moveIssueInKanbanList = []
    linkIssueToPullRequestList = []
    closeIssueList = []
   
    for issue in issues:
        createIssue = {}
        createIssue["timestamp"] = issue["created_at"]
        createIssue["personID"] = issue["user"]["login"]
        key = createIssue["personID"]
        createIssue["teamID"] ="n/a"
        if key in teamMembers:
            createIssue["teamID"] = teamMembers[createIssue["personID"]]
        createIssue["type"]="createIssue"
        createIssue["issueTitle"] = issue["title"]
        createIssue["issueID"] = issue["number"]
        createIssue["issueBody"] = issue["body"]
        createIssue["numCheckboxes"] = getCheckboxes(createIssue["issueBody"])
        createIssue["numCheckedCheckboxes"] = getCheckedCheckboxes(createIssue["issueBody"])
        createIssue["url"] = issue["html_url"]
        print("---------------------CREATE ISSUE-------------------------------------------------")
        print(createIssue)
        createIssueList.append(createIssue)
        
        
        response = requests.get(issue["comments_url"],headers=Headers,auth=Auth)
        comments = response.json()
        for comment in comments:
            commentOnIssue = {}
            commentOnIssue["timestamp"] = comment["created_at"]
            commentOnIssue["personID"] = comment["user"]["login"]
            key = commentOnIssue["personID"] 
            commentOnIssue["teamID"] ="n/a"
            if key in teamMembers:  
                commentOnIssue["teamID"] = teamMembers[commentOnIssue["personID"]]
            commentOnIssue["type"] = "commentOnIssue"
            commentOnIssue["issueID"] = issue["number"]
            commentOnIssue["commentBody"] = comment["body"]
            commentOnIssue["lastUpdated"] = comment["updated_at"]
            commentOnIssue["numCheckboxes"] = getCheckboxes(commentOnIssue["commentBody"])
            commentOnIssue["numCheckedCheckboxes"] = getCheckedCheckboxes(commentOnIssue["commentBody"])
            commentOnIssue["url"] = issue["html_url"]
            print("-----------------------------commentOnIssue--------------------------------------")
            print(commentOnIssue)
            commentOnIssueList.append(commentOnIssue)
        
        response = requests.get(issue["events_url"],headers=Headers,auth=Auth)
        events = response.json()
        for event in events:
            if event["event"] == "labeled":
                addLabelToIssue = {}
                addLabelToIssue["timestamp"] = event["created_at"]
                addLabelToIssue["personID"] = event["actor"]["login"]
                key = addLabelToIssue["personID"] 
                addLabelToIssue["teamID"] = "n/a"
                if key in teamMembers:
                    addLabelToIssue["teamID"] = teamMembers[addLabelToIssue["personID"]]
                addLabelToIssue["type"] = "addLabelToIssue"
                addLabelToIssue["issueID"] = issue["number"]
                addLabelToIssue["labelName"] = event["label"]["name"]
                addLabelToIssue["url"] = issue["html_url"]
                print("----------------------------addLabelToIssue--------------------------------------")
                print(addLabelToIssue)
                addLabelToIssueList.append(addLabelToIssue)
            elif event["event"] == "assigned":
                assignIssueToPerson = {}
                assignIssueToPerson["timestamp"] = event["created_at"]
                assignIssueToPerson["personID"] = event["assigner"]["login"]
                key = assignIssueToPerson["personID"]
                assignIssueToPerson["teamID"] ="n/a"
                if key in teamMembers:
                    assignIssueToPerson["teamID"] = teamMembers[assignIssueToPerson["personID"]]
                assignIssueToPerson["type"] = "assignIssueToPerson"
                assignIssueToPerson["issueID"] = issue["number"]
                assignIssueToPerson["assignee"] = event["assignee"]["login"]
                assignIssueToPerson["url"] = issue["html_url"]
                print("-----------------------------assignIssueToPerson--------------------------------------")
                print(assignIssueToPerson)
                assignIssueToPersonList.append(assignIssueToPerson)
            elif event["event"] == "milestoned":
                assignIssueToMilestone = {}
                assignIssueToMilestone["timestamp"] = event["created_at"]
                assignIssueToMilestone["personID"] = event["actor"]["login"]
                key = assignIssueToMilestone["personID"]
                assignIssueToMilestone["teamID"] = "n/a"
                if key in teamMembers:
                    assignIssueToMilestone["teamID"] = teamMembers[assignIssueToMilestone["personID"]]
                assignIssueToMilestone["type"] = "assignIssueToMilestone"
                assignIssueToMilestone["issueID"] = issue["number"]
                assignIssueToMilestone["milestone"] = event["milestone"]["title"]
#                 print(json.dumps(event,indent=4))
                key = assignIssueToMilestone["milestone"]
                assignIssueToMilestone["milestoneID"] = key
                if key in milestones.keys():
                    assignIssueToMilestone["milestoneID"] = milestones[key]
                assignIssueToMilestone["url"] = issue["html_url"]
                print("-----------------------------assignIssueToMilestone--------------------------------------")
                print(assignIssueToMilestone)
                assignIssueToMilestoneList.append(assignIssueToMilestone)
            elif event["event"] == "added_to_project":
                addIssueToKanban ={}
                addIssueToKanban["timestamp"] = event["created_at"]
                addIssueToKanban["personID"] = event["actor"]["login"]
                key = addIssueToKanban["personID"]
                addIssueToKanban["teamID"] ="n/a"
                if key in teamMembers:
                    addIssueToKanban["teamID"] = teamMembers[addIssueToKanban["personID"]]
                addIssueToKanban["type"]= "addIssueToKanban"
                addIssueToKanban["issueID"] = issue["number"]
                addIssueToKanban["url"] = issue["html_url"]
                print("-----------------------------addIssueToKanban-------------------------------------")
                print(addIssueToKanban)
                addIssueToKanbanList.append(addIssueToKanban)
            elif event["event"]== "moved_columns_in_project":
                moveIssueInKanban={}
                moveIssueInKanban["timestamp"] = event["created_at"]
                moveIssueInKanban["personID"] = event["actor"]["login"]
                key = moveIssueInKanban["personID"]
                moveIssueInKanban["teamID"] = "n/a"
                if key in teamMembers:
                    moveIssueInKanban["teamID"] = teamMembers[moveIssueInKanban["personID"]]
                moveIssueInKanban["type"] = "moveIssueInKanban"
                moveIssueInKanban["issueID"] = issue["number"]
                moveIssueInKanban["url"] = issue["html_url"]
                print("-----------------------------MoveIssueInKanban-------------------------------------")
                print(moveIssueInKanban)
                moveIssueInKanbanList.append(moveIssueInKanban)
                # boardID and columnID are not referenced/accessible atm
            elif event["event"] == "mentioned":
                tagIssue={}
                tagIssue["timestamp"] = event["created_at"]
                tagIssue["personID"] = event["actor"]["login"]
                key  = tagIssue["personID"]
                tagIssue["teamID"]="n/a"
                if key in teamMembers:
                    tagIssue["teamID"] = teamMembers[tagIssue["personID"]]
                tagIssue["type"] = "tagIssue"
                tagIssue["issueID"] = issue["number"]
                tagIssue["url"] = issue["html_url"]
                print("-----------------------------Tag Issue--------------------------------------")
                print(tagIssue)
                tagIssueList.append(tagIssue)
            elif event["event"] == "connected":
                linkIssueToPullRequest = {}
                linkIssueToPullRequest["timestamp"] = event["created_at"]
                linkIssueToPullRequest["personID"] = event["actor"]["login"]
                key =  linkIssueToPullRequest["personID"]
                linkIssueToPullRequest["teamID"] ="n/a"
                if key in teamMembers:
                    linkIssueToPullRequest["teamID"] = teamMembers[linkIssueToPullRequest["personID"]]
                linkIssueToPullRequest["type"] = "linkIssueToPullRequest"
                linkIssueToPullRequest["issueID"] = issue["number"]
                linkIssueToPullRequest["url"] = issue["html_url"]
                print("-----------------------------linkIssueToPullRequest--------------------------------------")
                print(linkIssueToPullRequest)
                linkIssueToPullRequestList.append(linkIssueToPullRequest)
            elif event["event"] == "closed":
                closeIssue = {}
                closeIssue["timestamp"] = event["created_at"]
                closeIssue["personID"] = event["actor"]["login"]
                key = closeIssue["personID"] 
                closeIssue["teamID"] = "n/a"
                if key in teamMembers:
                    closeIssue["teamID"] = teamMembers[closeIssue["personID"]]
                closeIssue["type"] ="closeIssue"
                closeIssue["issueID"] = issue["number"]
                closeIssue["url"] = issue["html_url"]
                print("-----------------------------closeIssue--------------------------------------")
                print(closeIssue)
                closeIssueList.append(closeIssue)
    
    print(f'total created issues: {len(createIssueList)}')
    print(f'total comments: {len(commentOnIssueList)}')
    
    issueEvents = {"createIssue":createIssueList, 
                   "addLabelToIssue":addLabelToIssueList,
                   "assignIssueToPerson":assignIssueToPersonList,
                   "assignIssueToMilestone":assignIssueToMilestoneList,
                   "commentOnIssue":commentOnIssueList,
                   "tagIssue":tagIssueList, 
                   "addIssueToKanban":addIssueToKanbanList, 
                   "moveIssueInKanban":moveIssueInKanbanList,
                   "linkIssueToPullRequest":linkIssueToPullRequestList, 
                   "closeIssue":closeIssueList
                  }
    return issueEvents

issueData = getIssueData(Issues,milestones=milestoneIDs,teamMembers=team_members)                
                
            

In [ ]:
def getKanbansData(teamMembers=team_members,url=projects_url,Headers=project_api_headers,Auth=(user,token)):
    createKanbanList = []
    response = requests.get(url,headers=Headers,auth=(user,token))
    kanbans = response.json()
    print(f"Total project boards for the repo: {len(kanbans)}")
#     print(json.dumps(kanbans,indent=4))
    for kanban in kanbans:
        createKanban = {}
        createKanban["timestamp"] = kanban["created_at"]
        createKanban["personID"] = kanban["creator"]["login"]
        key = createKanban["personID"]
        createKanban["teamID"] = "n/a"
        if key in teamMembers:
            createKanban["teamID"] = teamMembers[createKanban["personID"]]
        createKanban["type"] = "createKanban"
        createKanban["boardName"] = kanban["name"]
        createKanban["boardNumber"] = kanban["number"]
        createKanban["url"] = kanban["html_url"]
        print("------------------------------createKanban------------------------------------")
        print(createKanban)
        createKanbanList.append(createKanban)
    return {"createKanban":createKanbanList}

kanbanData = getKanbansData(teamMembers=team_members,url=projects_url)

In [ ]:
total_pull_request_pages = get_total_pages(pulls_search_url)
all_pulls_URLs = get_All_URLs(pulls_url,params=parameters,limit=total_pull_request_pages)
def getPullRequestData(urls,milestones,teamMembers=team_members,params=parameters,Headers=url_headers,Auth=(user,token)):
    createPullRequestList = []
    linkPullRequestToMilestoneList = []
    requestReviewList = []
    performReviewList = []
    commentOnPullRequestList = []
    mergePullRequestList = []
    closePullRequestList = []
    for url in urls:
        response = requests.get(url,headers=Headers,params=parameters,auth=Auth)
        result = response.json()
        for pull_request in result:
            createPullRequest = {}
            createPullRequest["timestamp"] = pull_request["created_at"]
            createPullRequest["personID"] = pull_request["user"]["login"]
            key = createPullRequest["personID"]
            createPullRequest["teamID"] ="n/a"
            if key in teamMembers:
                createPullRequest["teamID"] = teamMembers[createPullRequest["personID"]]
            createPullRequest["type"] = "createPullRequest"
            createPullRequest["pullRequestTitle"] = pull_request["title"]
            createPullRequest["pullRequestID"] = pull_request["number"]
            createPullRequest["pullRequestDescription"] = pull_request["body"]
            createPullRequest["url"] = pull_request["html_url"]
            print("--------------------------CreatePullRequest--------------------------------------")
            print(createPullRequest)
            createPullRequestList.append(createPullRequest)
            
            response = requests.get(pull_request["comments_url"],headers=Headers,auth=Auth)
            comments = response.json()
            for comment in comments:
                commentOnPullRequest = {}
                commentOnPullRequest["timestamp"] = comment["created_at"]
                commentOnPullRequest["personID"] = comment["user"]["login"]
                commentOnPullRequest["teamID"] = "n/a"
                key = commentOnPullRequest["personID"]
                if key in teamMembers:
                        commentOnPullRequest["teamID"] = teamMembers[commentOnPullRequest["personID"]]
                commentOnPullRequest["type"] = "commentOnPullRequest"
                commentOnPullRequest["pullRequestID"] = pull_request["number"]
                commentOnPullRequest["commentBody"] = comment["body"]
                commentOnPullRequest["lastUpdated"] = comment["updated_at"]
                commentOnPullRequest["numCheckboxes"] = getCheckboxes(commentOnPullRequest["commentBody"])
                commentOnPullRequest["numCheckedCheckboxes"] = getCheckedCheckboxes(commentOnPullRequest["commentBody"])
                commentOnPullRequest["url"] = pull_request["html_url"]
                print("-----------------------------commentOnPullRequest--------------------------------------")
                print(commentOnPullRequest)
                commentOnPullRequestList.append(commentOnPullRequest)
            
            
            
            response = requests.get(pull_request["review_comments_url"],headers=Headers,auth=Auth)
            review_comments = response.json()
#             print(json.dumps(review_comments,indent=4))
            if len(review_comments) > 0:
                for review in review_comments:
                    performReview = {}
                    performReview["timestamp"] = review["created_at"]
                    performReview["personID"] = review["user"]["login"]
                    key = performReview["personID"]
                    performReview["teamID"] ="n/a"
                    if key in teamMembers:
                        performReview["teamID"] = teamMembers[performReview["personID"]]
                    performReview["type"] = "performReview"
                    performReview["pullRequestID"] = pull_request["number"]
                    performReview["reviewComment"] = review["body"]
                    performReview["url"] = pull_request["html_url"]
                    print("--------------------------PerformReview------------------------------")
                    print(performReview)
                    performReviewList.append(performReview)     
#             print(json.dumps(pull_request, indent=4))
            issue_url = pull_request["issue_url"]
            response = requests.get(issue_url,headers=Headers,auth=Auth)
            issue = response.json()
            events_url = issue["events_url"]
            response = requests.get(events_url,headers=Headers,auth=Auth)
            events = response.json()
            for event in events:
                if event["event"] == "milestoned":
                    linkPullRequestToMilestone = {}
                    linkPullRequestToMilestone["timestamp"] = event["created_at"]
                    linkPullRequestToMilestone["personID"] = event["actor"]["login"]
                    key = linkPullRequestToMilestone["personID"]
                    linkPullRequestToMilestone["teamID"] = "n/a"
                    if key in teamMembers:
                        linkPullRequestToMilestone["teamID"] = teamMembers[linkPullRequestToMilestone["personID"]]
                    linkPullRequestToMilestone["type"] = "linkPullRequestToMilestone"
                    linkPullRequestToMilestone["pullRequestID"] = pull_request["number"]
                    linkPullRequestToMilestone["milestone"] = event["milestone"]["title"]
                    key = linkPullRequestToMilestone["milestone"] 
                    linkPullRequestToMilestone["milestoneID"] = key
                    if key in milestones.keys():
                        linkPullRequestToMilestone["milestoneID"] = milestones[key]  
                    linkPullRequestToMilestone["url"] = pull_request["html_url"]
                    print("-----------------------linkPullRequestToMilestone------------------------")
                    print(linkPullRequestToMilestone)
                    linkPullRequestToMilestoneList.append(linkPullRequestToMilestone)
                elif event["event"] == "review_requested":
                    requestReview = {}
                    requestReview["timestamp"] =  event["created_at"]
                    requestReview["personID"] = event["review_requester"]["login"]
                    key = requestReview["personID"]
                    requestReview["teamID"] ="n/a"
                    if key in teamMembers:
                        requestReview["teamID"] = teamMembers[requestReview["personID"]]
                    requestReview["type"] = "requestReview"
                    requestReview["pullRequestID"] = pull_request["number"]
#                     print(json.dumps(event,indent=4))
                    key = "requested_reviewer"
                    if key in event.keys():
                        requestReview["reviewer"] = event["requested_reviewer"]["login"]
                    else:
                        key = "requested_team"
                        if key in event.keys():
                            requestReview["reviewer"] = event["requested_team"]["name"]
                    requestReview["url"] = pull_request["html_url"]
                    print("----------------------------RequestReview---------------------------------")
                    print(requestReview)
                    requestReviewList.append(requestReview)
                elif event["event"] == "merged":
                    mergePullRequest = {}
                    mergePullRequest["timestamp"] = event["created_at"]
                    mergePullRequest["personID"] = event["actor"]["login"]
                    key = mergePullRequest["personID"]
                    mergePullRequest["teamID"] ="n/a"
                    if key in teamMembers:
                        mergePullRequest["teamID"] = teamMembers[mergePullRequest["personID"]]
                    mergePullRequest["type"] = "mergePullRequest"
                    mergePullRequest["pullRequestID"] = pull_request["number"]
                    mergePullRequest["url"] = pull_request["html_url"]
                    print("----------------------------mergePullRequest---------------------------------")
                    print(mergePullRequest)
                    mergePullRequestList.append(mergePullRequest)
                elif event["event"] == "closed":
                    closePullRequest = {}
                    closePullRequest["timestamp"] = event["created_at"]
                    closePullRequest["personID"] = event["actor"]["login"]
                    key = closePullRequest["personID"]
                    closePullRequest["teamID"] = "n/a"
                    if key in teamMembers:
                        closePullRequest["teamID"] = teamMembers[closePullRequest["personID"] ]
                    closePullRequest["type"] = "closePullRequest"
                    closePullRequest["pullRequestID"] = pull_request["number"]
                    closePullRequest["url"] = pull_request["html_url"]
                    print("----------------------------closePullRequest---------------------------------")
                    print(closePullRequest)
                    closePullRequestList.append(closePullRequest)
    
    pullRequestEvents = { "createPullRequest": createPullRequestList,
                          "linkPullRequestToMilestone":linkPullRequestToMilestoneList,
                          "requestReview":requestReviewList,
                          "performReview":performReviewList,
                          "commentOnPullRequest":commentOnPullRequestList,
                          "mergePullRequest":mergePullRequestList,
                          "closePullRequest":closePullRequestList
                        }
    return pullRequestEvents
            
        
        
pullRequestData = getPullRequestData(urls= all_pulls_URLs, milestones=milestoneIDs,teamMembers=team_members)
    

In [ ]:
def getCommitsOnly(commits):
        with open(repo+".csv", mode="w+",encoding="utf-8",newline='') as file:
            writer = csv.writer(file, delimiter=",")
            writer.writerow(["timestamp","person","team","type"])
    
        with open(repo+".csv",mode="a",encoding="utf-8",newline='') as file:
            writer = csv.writer(file,delimiter=",")
            for key in commits:
                commitsArray = commits[key]
                for item in commitsArray:
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                                    item["type"],item["branch"],
                                     item["person"],"\'"+item["commitMessage"]+"'",
                                    item["commitAdditions"],item["commitDeletions"],
                                    item["commitFilesTouched"],item["url"]])
getCommitsOnly(commitData)

In [ ]:
def getCSVFile(commits, issues, kanbans, pulls, milestones):
    with open(repo+".csv", mode="w+",encoding="utf-8",newline='') as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(["timestamp","person","team","type"])
    
    with open(repo+".csv",mode="a",encoding="utf-8",newline='') as file:
        writer = csv.writer(file,delimiter=",")
        for key in commits:
            commitsArray = commits[key]
            for item in commitsArray:
                writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                                item["type"],item["branch"],
                                 item["person"],"\'"+item["commitMessage"]+"'",
                                item["commitAdditions"],item["commitDeletions"],
                                item["commitFilesTouched"],item["url"]])
        
        for key in issues:
            print(key)
            issuesArray = issues[key]
#             print(issuesArray)
            for item in issuesArray:
                if key=="createIssue":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"], "\'"+item["issueTitle"]+"\'",item["issueID"],
                    item["issueBody"],item["numCheckboxes"],
                    item["numCheckedCheckboxes"],item["url"]])
                    
                elif key=="addLabelToIssue":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],
                    item["labelName"],item["url"]])
                    
                elif key == "assignIssueToPerson":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],
                    item["assignee"],item["url"]])    
                    
                elif key == "assignIssueToMilestone":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],
                    "\'"+item["milestone"]+"\'",item["milestoneID"],item["url"]])
                    
                elif key == "commentOnIssue":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],
                    "\'"+item["commentBody"]+"\'",item["lastUpdated"],
                    item["numCheckboxes"],item["numCheckedCheckboxes"],item["url"]])
                
                elif key =="tagIssue":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],item["url"]]) 
                
                elif key == "addIssueToKanban":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],item["url"]]) 
                    
                elif key == "moveIssueInKanban":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],item["url"]])
                    
                elif key == "linkIssueToPullRequest":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],item["url"]])
                    
                elif key == "closeIssue":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["issueID"],item["url"]])
        
        for key in kanbans:
            print(key)
            kanbansArray = kanbans[key]
#             print(issuesArray)
            for item in kanbansArray:
                if key=="createKanban":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["boardName"],item["boardNumber"],item["url"]])
              
        for key in pulls:
            print(key)
            pullsArray = pulls[key]

            for item in pullsArray:
                if key=="createPullRequest":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],"\'"+item["pullRequestTitle"]+"\'",item["pullRequestID"],
                    "\'"+item["pullRequestDescription"]+"\'",item["url"]])
                    
                elif key=="linkPullRequestToMilestone":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["pullRequestID"],
                    item["milestone"],item["milestoneID"],item["url"]])
                    
                elif key == "requestReview":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["pullRequestID"],
                    item["reviewer"],item["url"]])    
                    
                elif key == "performReview":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["pullRequestID"],
                    "\'"+item["reviewComment"]+"\'",item["url"]])
                    
                elif key == "commentOnPullRequest":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["pullRequestID"],
                    "\'"+item["commentBody"]+"\'",item["lastUpdated"],
                    item["numCheckboxes"],item["numCheckedCheckboxes"],item["url"]])
                
                elif key =="mergePullRequest":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["pullRequestID"],item["url"]]) 
                
                elif key == "closePullRequest":
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],item["pullRequestID"],item["url"]]) 
                    
            
        for key in milestones:
            print(key)
            milestonesArray = milestones[key]
#             print(issuesArray)
            for item in milestonesArray:
                if key=="createMilestone":
                    if item["milestoneDescription"]==None:
                        item["milestoneDescription"] = ""
                    writer.writerow([item["timestamp"],item["personID"],item["teamID"],
                    item["type"],"\'"+item["title"]+"\'",item["milestoneID"],
                                     "\'"+item["milestoneDescription"]+"\'",item["url"]])
            

getCSVFile(commits=commitData, issues=issueData, kanbans=kanbanData, pulls=pullRequestData,
              milestones=milestoneData)

In [ ]:
def main(Org=org, Repo=repo):
    projects_url,teams_url,commits_url,issues_search_url,issues_url,pulls_search_url,pulls_url,milestones_url,branches_url= makeURLs(Org,Repo)
    team_members,memberIDToName = getTeamMembers(teamUrl=teams_url)
    Branches = get_branches(url=branches_url)
    print(Branches)
    print("\nCOMMITS\n")
    BranchCommits= get_branch_commit_urls(commits_url,Branches)
    commitData = get_commit_data(commitBranches=BranchCommits,teamMembers=team_members,names=memberIDToName)
#     commitPages = get_total_pages(commits_url)
#     print(f'total pages to process: {commitPages}')
#     all_commit_urls = get_All_URLs(commits_url,commitPages)
#     commitLinks = get_commit_urls(all_commit_urls)
#     print(f'total commit items to scrape: {len(commitLinks)}')
#     commitData = getCommitData(commitLinks,teamMembers=team_members)
      
    milestoneData,milestoneIDs = getMilestonesData(teamMembers=team_members,url=milestones_url)
    print("\nISSUES\n")
    total_issue_pages = get_total_pages(issues_search_url)
    all_issue_URLs = get_All_URLs(issues_url,total_issue_pages,params=parameters)
    Issues,Pulls = separate_issues_from_pulls(urls=all_issue_URLs)
    issueData = getIssueData(Issues,milestones=milestoneIDs,teamMembers=team_members)                
    print("\KANBANS\n")
    kanbanData = getKanbansData(teamMembers=team_members,url=projects_url)
    print("\nPULL REQUESTS\n")
    total_pull_request_pages = get_total_pages(pulls_search_url)
    all_pulls_URLs = get_All_URLs(pulls_url,total_pull_request_pages,params=parameters)
    pullRequestData = getPullRequestData(urls= all_pulls_URLs, milestones=milestoneIDs,teamMembers=team_members)
    getCSVFile(commits=commitData, issues=issueData, kanbans=kanbanData, pulls=pullRequestData,
              milestones=milestoneData)

    
# main()


In [ ]:
file, ex = os.path.splitext(".gitignore")
